In [1]:
# -*- coding: utf-8 -*-

import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import wave
import librosa

In [2]:
datadir = "../data/"
learningdir = "LearningData/magiarecord/"

breakpoint_file = "breakpoint.csv"

In [3]:
def loadDivideLine(filepath):
    print(filepath)
    #音声の区切り位置読み込み
    #[開始時刻、終了時刻、人物ID]
    if not os.path.exists(filepath):
        return -1
    
    divideline = []
    with open(filepath, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            divideline.append(row)
    divideline = np.array(divideline,dtype='float64')
            
    return divideline

In [4]:
def readWave(filepath):
    wf = wave.open(filepath, "r")
    fs = wf.getframerate()
    x = wf.readframes(wf.getnframes())
    x = np.frombuffer(x, dtype="int16") / 32768.0  # (-1, 1)に正規化
    wf.close()
    return x, float(fs)

In [5]:
def loadWavefile(filename,rate=44100):
    y,sr = librosa.load(filename,sr=rate)
    return y,sr

def outputWavefile(filepath,y,sr=44100):
    librosa.output.write_wav(filepath,y,sr)

In [6]:
def divideVoiceData(divedefilepath,wavpath):
    
    #load divide_time data
    dividepoint = loadDivideLine(divedefilepath)
    if np.min(dividepoint) < 0:
        print('divide_time data is nothing')
        return -1
    divide_sec = 60*dividepoint.T[0]+dividepoint.T[1]
    peoplelist = np.array(dividepoint.T[2],dtype='int32')
    
    diff = divide_sec[1:]-divide_sec[:-1]
    if min(diff) < 0:
        print('divide_time value is wrong')
        return -1
    
    #make directory
    number_of_data = {}
    directory_path = {}
    for people in set(peoplelist):
        people = int(people)
        if people == 0:
            continue
        subprocess.call('mkdir '+datadir+learningdir+str(people))
        number_of_data[people] = len(os.listdir(datadir+learningdir+str(people)))
        directory_path[people] = datadir+learningdir+str(people)
        
    #load wav data
    print('Loading wav data')
    wave, fs = loadWavefile(wavpath)
    
    last_time = 0.0
    for time,people in zip(divide_sec,peoplelist):
        if people == 0:
            last_time = time
            continue
            
        s = int(last_time * fs)
        e = int(time * fs)
        split_data_path = directory_path[people]+'/'+str(number_of_data[people])+'.wav'
        print(last_time,people,split_data_path)
        wave_part = wave[s:e]
        outputWavefile(split_data_path,wave_part)
            
        number_of_data[people] += 1
        last_time = time
        
    return 1

In [7]:
csvpath = "../data/LearningData/magiarecord/main/main_1-1.csv"
wavpath = "../data/LearningData/magiarecord/main/main_1-1.wav"

In [8]:
#divideVoiceData(csvpath,wavpath)

../data/LearningData/magiarecord/main/main_1-1.csv
Loading wav data
3.0 1 ../data/LearningData/magiarecord/1/0.wav
53.0 1 ../data/LearningData/magiarecord/1/1.wav
103.0 1 ../data/LearningData/magiarecord/1/2.wav
140.0 1 ../data/LearningData/magiarecord/1/3.wav
154.0 6 ../data/LearningData/magiarecord/6/0.wav
156.0 1 ../data/LearningData/magiarecord/1/4.wav
168.0 6 ../data/LearningData/magiarecord/6/1.wav
178.0 1 ../data/LearningData/magiarecord/1/5.wav
180.0 6 ../data/LearningData/magiarecord/6/2.wav
184.5 1 ../data/LearningData/magiarecord/1/6.wav
189.3 6 ../data/LearningData/magiarecord/6/3.wav
198.5 6 ../data/LearningData/magiarecord/6/4.wav
201.0 1 ../data/LearningData/magiarecord/1/7.wav
207.0 6 ../data/LearningData/magiarecord/6/5.wav
212.5 1 ../data/LearningData/magiarecord/1/8.wav
219.4 6 ../data/LearningData/magiarecord/6/6.wav
226.2 1 ../data/LearningData/magiarecord/1/9.wav
229.0 6 ../data/LearningData/magiarecord/6/7.wav
238.3 1 ../data/LearningData/magiarecord/1/10.wav
240

1